In [1]:
set.seed(123)

n      <- 6                  # tamanho de amostra do exercício
theta0 <- 2.7                # "verdadeiro" theta
g0     <- exp(-theta0)       # valor verdadeiro de g(theta) = P(Z>1)

B <- 10000                   # número de simulações
covered <- logical(B)        # vetor lógico para registrar cobertura

# quantis qui-quadrado usados sempre (df = 2n)
df  <- 2 * n
qlo <- qchisq(0.025, df = df)
qhi <- qchisq(0.975, df = df)

for (b in 1:B) {
  # 1) gera amostra Exp(theta0)
  z <- rexp(n, rate = theta0)
  
  # 2) soma
  S <- sum(z)
  
  # 3) IC para theta
  L_theta <- qlo / (2 * S)
  U_theta <- qhi / (2 * S)
  
  # 4) IC para g(theta) = exp(-theta)
  L_g <- exp(-U_theta)
  U_g <- exp(-L_theta)
  
  # 5) verifica se g0 está dentro
  covered[b] <- (L_g <= g0) && (g0 <= U_g)
}

# estimativa da cobertura
mean(covered)

[1] 0.955

In [7]:
set.seed(123)

# Parâmetros do experimento
theta0   <- 2.703     # "verdadeiro" theta
n        <- 6         # tamanho da amostra
M        <- 1000     # número de simulações
z_0975   <- 1.96      # quantil 0.975 da N(0,1)

# valor verdadeiro de g(theta) = P(Z > 1)
g_true <- exp(-theta0)

# vetor para guardar se o IC cobriu ou não
covered <- logical(M)

for (m in 1:M) {
  # 1) cria uma amostra com distribuição Exponencial(theta0)
  z <- rexp(n, theta0)
  
  # 2) EMV de theta: 1 / media
  theta_hat <- 1 / mean(z)
  
  # 3) IC aproximado para theta
  L_theta <- theta_hat * (1 - z_0975 / sqrt(n))
  U_theta <- theta_hat * (1 + z_0975 / sqrt(n))
  
  # 4) transforma para g(theta) = e^{-theta}
  L_g <- exp(-U_theta)
  U_g <- exp(-L_theta)
  
  # 5) verifica se g_true está no intervalo
  covered[m] <- (L_g <= g_true) && (g_true <= U_g)
}

# Estimativa da cobertura
mean(covered)

[1] 0.962

In [2]:
set.seed(123)

theta0 <- 2        # valor verdadeiro
n      <- 6       # tamanho amostral inicial
B      <- 10000    # número de repetições

cover_a <- cover_b <- cover_c <- 0

for (b in 1:B) {
  # 1) Amostra exponencial
  z  <- rexp(n, rate = theta0)   # atenção: rate = theta
  zbar <- mean(z)
  theta_hat <- 1 / zbar
  
  # 2) Variância assintótica de theta_hat
  var_theta_hat <- theta_hat^2 / n
  
  ## --- (a) g(theta) = P(Z > 1) = exp(-theta)
  g_a_hat  <- exp(-theta_hat)
  g_a_der  <- -exp(-theta_hat)
  var_g_a  <- (g_a_der^2) * var_theta_hat
  sd_g_a   <- sqrt(var_g_a)
  IC_a     <- c(g_a_hat - 1.96*sd_g_a,
                g_a_hat + 1.96*sd_g_a)
  g_a_true <- exp(-theta0)
  cover_a  <- cover_a + (g_a_true >= IC_a[1] &&
                           g_a_true <= IC_a[2])
  
  ## --- (b) g(theta) = P(0.1 < Z < 1)
  g_b_hat  <- exp(-0.1*theta_hat) - exp(-theta_hat)
  g_b_der  <- -0.1*exp(-0.1*theta_hat) + exp(-theta_hat)
  var_g_b  <- (g_b_der^2) * var_theta_hat
  sd_g_b   <- sqrt(var_g_b)
  IC_b     <- c(g_b_hat - 1.96*sd_g_b,
                g_b_hat + 1.96*sd_g_b)
  g_b_true <- exp(-0.1*theta0) - exp(-theta0)
  cover_b  <- cover_b + (g_b_true >= IC_b[1] &&
                           g_b_true <= IC_b[2])
  
  ## --- (c) g(theta) = Var(Z) = 1/theta^2
  g_c_hat  <- 1 / theta_hat^2
  g_c_der  <- -2 / theta_hat^3
  var_g_c  <- (g_c_der^2) * var_theta_hat
  sd_g_c   <- sqrt(var_g_c)
  IC_c     <- c(g_c_hat - 1.96*sd_g_c,
                g_c_hat + 1.96*sd_g_c)
  g_c_true <- 1 / theta0^2
  cover_c  <- cover_c + (g_c_true >= IC_c[1] &&
                           g_c_true <= IC_c[2])
}

cover_a <- cover_a / B
cover_b <- cover_b / B
cover_c <- cover_c / B
cover_a; cover_b; cover_c

[1] 0.8378

[1] 0.9379

[1] 0.8323